# Homework 2 - balancing & binning (deadline 28. 11. 2020)

In short, the main task is to play with balancing and binning to obtain the best results for the binary classification task.
  
> The instructions are not given in details: It is up to you to come up with ideas on how to fulfill the particular tasks as best you can!

## What are you supposed to do:

  1. Download the dataset [here](https://www.dropbox.com/s/aq08ytozfplb97b/data.csv?dl=0).
  1. Use 2 binning methods (on features of your choice, with your choice of parameters) and comment on its effects on classification performance.
  1. Use at least 2 other data balancing techniques of your choice on the dataset and comment the classification results. **Just copied code from tutorial 4 will not be accepted.**
  1. Run all classification tests at least three times - once for unbalanced original data, twice for balanced data (try at least 2 balancing techniques), compare those results (give a comment).
  
Give comments (!) on each step of your solution, with short explanations of your choices.

**If you do all this properly, you will obtain 16 points.** 

## Comments

  * Please follow the instructions from https://courses.fit.cvut.cz/MI-PDD/homeworks/index.html.
  * If the reviewing teacher is not satisfied, she can (!) give you another chance to rework your homework and to obtain more points. However, this is not a given, so do your best! :)
  * English is not compulsory.
  
## Data description

* The dataset can be downloaded [here](https://www.dropbox.com/s/aq08ytozfplb97b/data.csv?dl=0).
* The data are devoted to the binary classification task, the aim is to predict the probability that a driver will initiate an auto insurance claim in the next year.
* Target feature is called 'y' and signifies whether or not a claim was filed for that policy holder.
* To fulfill the task one does not need to know the meaning of predictors.
* Predictors that belong to similar groupings are tagged as such in the feature names (e.g., ind, reg, car, calc). In addition, feature names include the postfix bin to indicate binary features and cat to indicate categorical features. Features without these designations are either continuous or ordinal. Values of -1 indicate that the feature was missing from the observation.
* While using [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html), control the shuffling of data by random_state parameter. Do not use shuffle=False, probably never (can cause systematic error).

In [56]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_log_error, make_scorer, mean_squared_error, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split, cross_val_score

In [57]:
USE_PRECALCULED = True

In [58]:
df = pd.read_csv('data.csv')

In [59]:
display(df.sample(5))
display(df.describe())

,y,xind_01,xind_02_cat,xind_03,xind_04_cat,xind_05_cat,xind_06_bin,xind_07_bin,xind_08_bin,xind_09_bin,...,xcalc_11,xcalc_12,xcalc_13,xcalc_14,xcalc_15_bin,xcalc_16_bin,xcalc_17_bin,xcalc_18_bin,xcalc_19_bin,xcalc_20_bin
469726,0,3,1,9,0,0,1,0,0,0,...,4,2,6,5,0,1,1,1,0,0
36552,0,1,1,8,0,0,1,0,0,0,...,6,1,3,10,1,1,1,0,1,0
250120,0,0,1,3,0,0,1,0,0,0,...,6,0,4,9,0,0,1,0,0,0
215010,0,2,1,3,1,0,0,1,0,0,...,4,2,4,5,0,1,1,0,1,0
42384,0,1,1,8,0,0,1,0,0,0,...,2,4,4,9,0,1,1,0,1,0


,y,xind_01,xind_02_cat,xind_03,xind_04_cat,xind_05_cat,xind_06_bin,xind_07_bin,xind_08_bin,xind_09_bin,...,xcalc_11,xcalc_12,xcalc_13,xcalc_14,xcalc_15_bin,xcalc_16_bin,xcalc_17_bin,xcalc_18_bin,xcalc_19_bin,xcalc_20_bin
count,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,...,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000,595212.000000
mean,0.036448,1.900378,1.358943,4.423318,0.416794,0.405188,0.393742,0.257033,0.163921,0.185304,...,5.441382,1.441918,2.872288,7.539026,0.122427,0.627840,0.554182,0.287182,0.349024,0.153318
std,0.187401,1.983789,0.664594,2.699902,0.493311,1.350642,0.488579,0.436998,0.370205,0.388544,...,2.332871,1.202963,1.694887,2.746652,0.327779,0.483381,0.497056,0.452447,0.476662,0.360295
min,0.000000,0.000000,-1.000000,0.000000,-1.000000,-1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,4.000000,1.000000,2.000000,6.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,1.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,5.000000,1.000000,3.000000,7.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000
75%,0.000000,3.000000,2.000000,6.000000,1.000000,0.000000,1.000000,1.000000,0.000000,0.000000,...,7.000000,2.000000,4.000000,9.000000,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000
max,1.000000,7.000000,4.000000,11.000000,1.000000,6.000000,1.000000,1.000000,1.000000,1.000000,...,19.000000,10.000000,13.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


## Binning

Use 2 binning methods (on features of your choice, with your choice of parameters) and comment on its effects on classification performance.

Binning is used for continuous variable discretization. 

### Detect the variables to apply binning to

In [60]:
continuos_cols_mask = [not ('bin' in x or 'cat' in x or x=='y') for x in df.columns]

In [61]:
df.columns[continuos_cols_mask]

Index(['xind_01', 'xind_03', 'xind_14', 'xind_15', 'xreg_01', 'xreg_02',
       'xreg_03', 'xcar_11', 'xcar_12', 'xcar_13', 'xcar_14', 'xcar_15',
       'xcalc_01', 'xcalc_02', 'xcalc_03', 'xcalc_04', 'xcalc_05', 'xcalc_06',
       'xcalc_07', 'xcalc_08', 'xcalc_09', 'xcalc_10', 'xcalc_11', 'xcalc_12',
       'xcalc_13', 'xcalc_14'],
      dtype='object')

In [62]:
def num_bins(c):
    uniques = c.nunique()
    # Square root choice
    return int(np.ceil(uniques**(1/2)))

def get_binning(df, cols, method):
    for c in cols:
        n = num_bins(df[c])
        df[c] = method(df[c], n, duplicates='drop').cat.codes
    return df

In [63]:
df_c = df.copy()
cut = get_binning(df_c, df_c.columns[continuos_cols_mask], pd.cut)
df_c = df.copy()
qcut = get_binning(df_c, df_c.columns[continuos_cols_mask], pd.qcut)

In [64]:
clf = DecisionTreeClassifier()

In [65]:
X = df.drop('y', axis=1)
y = df.y

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=0)
y_pred = clf.fit(X_train, y_train).predict(X_test)
print('\nOriginal data \nAccuracy: ', accuracy_score(y_test, y_pred),'\nF1-score: ', f1_score(y_test, y_pred))

X = cut.drop('y', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=0)
y_pred = clf.fit(X_train, y_train).predict(X_test)
print('Cut data \nAccuracy: ', accuracy_score(y_test, y_pred),'\nF1-score: ', f1_score(y_test, y_pred))

X = qcut.drop('y', axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=0)
y_pred = clf.fit(X_train, y_train).predict(X_test)
print('Qcut data \nAccuracy: ', accuracy_score(y_test, y_pred),'\nF1-score: ', f1_score(y_test, y_pred))


Original data 
Accuracy:  0.9190835205913854 
F1-score:  0.05382839742645253
Cut data 
Accuracy:  0.9157065753827415 
F1-score:  0.056952210892345284
Qcut data 
Accuracy:  0.9149379423315203 
F1-score:  0.05496966868875408


We used 2 binning methods 
<li><b>cut</b>, which is method to split data into bins with equal width
<li><b>qcut</b>, which is method to split data into bins with equal depth
As bins count, we used <b> Square root choice</b> method 
We can see, that the F-1 score is really low, this is probably because the data are unbalanced (the distribution of y is far from 50/50), we will check this in part two.

The f-score is overally low, but we can see slight improvement with the "cut" approach. We might achieve better score if we changed categorical data to one hot encoding.

## Data balancing
Use at least 2 other data balancing techniques of your choice on the dataset and comment the classification results. Just copied code from tutorial 4 will not be accepted.

In [66]:
total = df.y.count()
pos = df[df.y == 0].y.count()
neg = df[df.y == 1].y.count()
print(f'Distribution in target var: \n 1 - {round(100*pos/total, 1)} % \n 0 - {round(100*neg/total, 1)} %')

Distribution in target var: 
 1 - 96.4 % 
 0 - 3.6 %


We will make use of following methods, first two as undersampling and the third as oversampling
- One-side Sampling (OSS)
- Neighborhood Cleaning Rule (NCL) 
- SMOTE
- Random over sampling

### One-side Sampling

In [67]:
from collections import Counter

In [68]:
if not USE_PRECALCULED:
    from imblearn.under_sampling import OneSidedSelection
    oss = OneSidedSelection()
    X = df.drop('y', axis=1)
    y = df.y

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=0)
    y_pred = oss.fit_sample(X_train, y_train)
    Counter(y_pred[1])
    clf.fit(y_pred[0], y_pred[1])
    predict = clf.predict(X_test)
    f1 = f1_score(y_test, predict)
    print(f'\nF1 score: {f1}')
else:
    print(f'\nF1 score: 0.05188243866893369')


F1 score: 0.05188243866893369


### Neighbourhood cleaning

In [69]:
if not USE_PRECALCULED: 
    from imblearn.under_sampling import NeighbourhoodCleaningRule
    ncl = NeighbourhoodCleaningRule()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=0)
    y_pred = ncl.fit_sample(X_train, y_train)
    print(Counter(y_pred[1]))
    clf.fit(y_pred[0], y_pred[1])
    predict = clf.predict(X_test)
    f1 = f1_score(y_test, predict)
    print(f'\nF1 score: {f1}')
else:
    print('Counter({0: 309880, 1: 12844})')
    print('F1 score: 0.05619069659800972')

Counter({0: 309880, 1: 12844})
F1 score: 0.05619069659800972


### SMOTE

In [70]:
if not USE_PRECALCULED:
    from imblearn.over_sampling import SMOTE
    smt = SMOTE()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=0)
    y_pred = smt.fit_sample(X_train, y_train)
    print(Counter(y_pred[1]))
else:
    print('Counter({0: 344283, 1: 344283})')

Counter({0: 344283, 1: 344283})


In [72]:
if not USE_PRECALCULED:
    clf.fit(y_pred[0], y_pred[1])
    predict = clf.predict(X_test)
    f1 = f1_score(y_test, predict)
    print(f'\nF1 score: {f1}')
else:
    print(f'\nF1 score: 0.058004640371229696')


F1 score: 0.058004640371229696


### Random over sampling

In [73]:
if not USE_PRECALCULED:
    from imblearn.over_sampling import RandomOverSampler
    ros = RandomOverSampler()
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.4, random_state=0)
    y_pred = ros.fit_sample(X_train, y_train)
    print(Counter(y_pred[1]))
    clf.fit(y_pred[0], y_pred[1])
    predict = clf.predict(X_test)
    f1 = f1_score(y_test, predict)
    print(f'\nF1 score: {f1}')
else:
    print('Counter({0: 344283, 1: 344283})')
    print('F1 score: 0.04985176238058636')

Counter({0: 344283, 1: 344283})
F1 score: 0.04985176238058636


Apparently, best method for this imbalanced dataset is SMOTE. This dataset was very imbalanced overall, thus the f score is really low.